*Curso: Big Data - Métodos Intensivos de Computación*</br>
*Presentado por: Boris Mauricio Martinez Gutierrez*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/BigData/dask/

Mounted at /content/drive
/content/drive/MyDrive/BigData/dask


In [ ]:
# instala la version mas reciente de dask
!pip install distributed==2021.4.0

Github Archive
----------------

[Download link](https://www.githubarchive.org/)

Every public github event for the last few years stored as gzip compressed line-delimited JSON data.  Watch out, the schema switches at the 2014-2015 transition.

**Try the following:**

*  Use `dask.bag` to inspect the data
*  Drill down using functions like `pluck` and `filter`
*  Find who the most popular committers were in 2015

In [ ]:
# descarga del dataset en el directorio asignado
import os

path = os.path.join('data', 'git_archive')
if not os.path.exists(path):
  os.makedirs(path)

path

'data/git_archive'

In [ ]:
# datos por hora del año 2015
!wget -P data/git_archive -c https://data.gharchive.org/2015-{01..12}-{01..31}-{0..23}.json.gz

Se han truncado las últimas 5000 líneas del flujo de salida.
Length: 7423677 (7.1M) [application/gzip]
Saving to: ‘data/git_archive/2015-12-11-4.json.gz’

2015-12-11-4.json.g 100%[===================>]   7.08M  7.85MB/s    in 0.9s    

2021-05-09 20:29:07 (7.85 MB/s) - ‘data/git_archive/2015-12-11-4.json.gz’ saved [7423677/7423677]

--2021-05-09 20:29:07--  https://data.gharchive.org/2015-12-11-5.json.gz
Reusing existing connection to data.gharchive.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 7184227 (6.9M) [application/gzip]
Saving to: ‘data/git_archive/2015-12-11-5.json.gz’

2015-12-11-5.json.g 100%[===================>]   6.85M  12.1MB/s    in 0.6s    

2021-05-09 20:29:09 (12.1 MB/s) - ‘data/git_archive/2015-12-11-5.json.gz’ saved [7184227/7184227]

--2021-05-09 20:29:09--  https://data.gharchive.org/2015-12-11-6.json.gz
Reusing existing connection to data.gharchive.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 8386330 (8.0M) [application/gzip]

In [ ]:
# tamaño en disco del dataset
!du -sh data/git_archive

74G	data/git_archive


In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
import dask.bag as db

lines = db.read_text(os.path.join(path, '2015*.json.gz'))
lines.take(1)

('{"id":"2489368070","type":"PushEvent","actor":{"id":9152315,"login":"davidjhulse","gravatar_id":"","url":"https://api.github.com/users/davidjhulse","avatar_url":"https://avatars.githubusercontent.com/u/9152315?"},"repo":{"id":28635890,"name":"davidjhulse/davesbingrewardsbot","url":"https://api.github.com/repos/davidjhulse/davesbingrewardsbot"},"payload":{"push_id":536740396,"size":1,"distinct_size":1,"ref":"refs/heads/master","head":"a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81","before":"86ffa724b4d70fce46e760f8cc080f5ec3d7d85f","commits":[{"sha":"a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81","author":{"email":"da8d7d1118ca5befd4d0d3e4f449c76ba6f1ee7e@live.com","name":"davidjhulse"},"message":"Altered BingBot.jar\\n\\nFixed issue with multiple account support","distinct":true,"url":"https://api.github.com/repos/davidjhulse/davesbingrewardsbot/commits/a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81"}]},"public":true,"created_at":"2015-01-01T00:00:00Z"}\n',)

In [ ]:
import json
js = lines.map(json.loads)
# inspeccionamos el primer elemento
js.take(1)

({'actor': {'avatar_url': 'https://avatars.githubusercontent.com/u/9152315?',
   'gravatar_id': '',
   'id': 9152315,
   'login': 'davidjhulse',
   'url': 'https://api.github.com/users/davidjhulse'},
  'created_at': '2015-01-01T00:00:00Z',
  'id': '2489368070',
  'payload': {'before': '86ffa724b4d70fce46e760f8cc080f5ec3d7d85f',
   'commits': [{'author': {'email': 'da8d7d1118ca5befd4d0d3e4f449c76ba6f1ee7e@live.com',
      'name': 'davidjhulse'},
     'distinct': True,
     'message': 'Altered BingBot.jar\n\nFixed issue with multiple account support',
     'sha': 'a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81',
     'url': 'https://api.github.com/repos/davidjhulse/davesbingrewardsbot/commits/a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81'}],
   'distinct_size': 1,
   'head': 'a9b22a6d80c1e0bb49c1cf75a3c075b642c28f81',
   'push_id': 536740396,
   'ref': 'refs/heads/master',
   'size': 1},
  'public': True,
  'repo': {'id': 28635890,
   'name': 'davidjhulse/davesbingrewardsbot',
   'url': 'https://ap

In [ ]:
# interesan los registros donde un usuario da una estrella a un repositorio
js_filtered = js.filter(lambda record: record['type'] == 'WatchEvent')
js_filtered.take(2)

({'actor': {'avatar_url': 'https://avatars.githubusercontent.com/u/642929?',
   'gravatar_id': '',
   'id': 642929,
   'login': 'jchristi',
   'url': 'https://api.github.com/users/jchristi'},
  'created_at': '2015-01-01T00:00:18Z',
  'id': '2489368319',
  'org': {'avatar_url': 'https://avatars.githubusercontent.com/u/7116887?',
   'gravatar_id': '',
   'id': 7116887,
   'login': 'LinuxStandardBase',
   'url': 'https://api.github.com/orgs/LinuxStandardBase'},
  'payload': {'action': 'started'},
  'public': True,
  'repo': {'id': 18297319,
   'name': 'LinuxStandardBase/lsb',
   'url': 'https://api.github.com/repos/LinuxStandardBase/lsb'},
  'type': 'WatchEvent'},
 {'actor': {'avatar_url': 'https://avatars.githubusercontent.com/u/1405427?',
   'gravatar_id': '',
   'id': 1405427,
   'login': 'tategakibunko',
   'url': 'https://api.github.com/users/tategakibunko'},
  'created_at': '2015-01-01T00:00:24Z',
  'id': '2489368389',
  'payload': {'action': 'started'},
  'public': True,
  'repo': 

In [ ]:
# nombre de usuario y nombre del repositorio en el cual colocó una estrella
def stared_repos(d):
  try:
    return {'user': d['actor']['login'], 'stared_repo': d['repo']['name']}
  except KeyError:
    return {'user': '', 'stared_repo': ''}
          
js_filtered.map(stared_repos).take(5)

({'stared_repo': 'LinuxStandardBase/lsb', 'user': 'jchristi'},
 {'stared_repo': 'inf0rmer/blanket', 'user': 'tategakibunko'},
 {'stared_repo': 'parrt/cs652', 'user': 'alfateam123'},
 {'stared_repo': 'Pathgather/please-wait', 'user': 'senthilnathansk'},
 {'stared_repo': 'jtsternberg/Shortcode_Button', 'user': 'teamcrisis'})

In [ ]:
# lista de los repositorios más populares en términos de seguidores (estrellas)
js_filtered.map(stared_repos).pluck('stared_repo').take(5)

('LinuxStandardBase/lsb',
 'inf0rmer/blanket',
 'parrt/cs652',
 'Pathgather/please-wait',
 'jtsternberg/Shortcode_Button')

In [ ]:
# top 100 repositorios más populares en términos de seguidores (estrellas)
from dask.diagnostics import ProgressBar

with ProgressBar():
  top_repos = (js_filtered.map(stared_repos).pluck('stared_repo').frequencies().topk(100, key=1)).compute()

top_repos[:10]

[('FreeCodeCamp/FreeCodeCamp', 38485),
 ('facebook/react-native', 25888),
 ('apple/swift', 25834),
 ('sindresorhus/awesome', 24418),
 ('facebook/react', 22971),
 ('jlevy/the-art-of-command-line', 22105),
 ('NARKOZ/hacker-scripts', 20450),
 ('twbs/bootstrap', 19768),
 ('google/material-design-lite', 17904),
 ('airbnb/javascript', 17586)]

In [ ]:
# el resultado ahora puede manejarse con pandas
import pandas as pd

df = pd.DataFrame(top_repos, columns=['repository', 'stars_count'])
df.head()

,repository,stars_count
0,FreeCodeCamp/FreeCodeCamp,38485
1,facebook/react-native,25888
2,apple/swift,25834
3,sindresorhus/awesome,24418
4,facebook/react,22971
...,...,...
95,Automattic/wp-calypso,6581
96,Polymer/polymer,6574
97,vsouza/awesome-ios,6518
98,ipader/SwiftGuide,6515


In [ ]:
client.close()